# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [3]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [4]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

cur.execute(
    """
    SELECT *
    FROM sqlite_master
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,type,name,tbl_name,rootpage,sql
0,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
1,table,offices,offices,32,"CREATE TABLE ""offices"" (\n\t""officeCode""\tINTE..."
2,table,customers,customers,57,"CREATE TABLE ""customers"" (\n\t""customerNumber""..."
3,table,employees,employees,35,"CREATE TABLE ""employees"" (\n\t""employeeNumber""..."
4,table,orderdetails,orderdetails,2,"CREATE TABLE ""orderdetails"" (\n\t""orderNumber""..."
5,table,orders,orders,27,"CREATE TABLE ""orders"" (\n\t""orderNumber""\tINTE..."
6,table,payments,payments,28,"CREATE TABLE ""payments"" (\n\t""customerNumber""\..."
7,table,products,products,4,"CREATE TABLE ""products"" (\n\t""productCode""\tTE..."


## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [5]:
# Your code here; use a subquery. No join is necessary 


cur.execute(
    """
    SELECT customerNumber, contactLastName, contactFirstName
    FROM customers
    WHERE customerNumber IN (SELECT customerNumber
                            FROM orders
                            WHERE orderDate = '2003-01-31'

                            )
    
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [6]:
# Your code here
cur.execute(
    """
    SELECT productCode, SUM(quantityordered)
    FROM orderdetails
    WHERE productCode IN (SELECT productCode
                            FROM products
                            )
    GROUP BY productCode
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productCode,SUM(quantityordered)
0,S10_1678,1057
1,S10_1949,961
2,S10_2016,999
3,S10_4698,985
4,S10_4757,1030
...,...,...
104,S700_3505,952
105,S700_3962,896
106,S700_4002,1085
107,S72_1253,960


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [13]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute(
    """
    SELECT productName, COUNT(customerNumber) as count
    FROM (SELECT DISTINCT productName, customerNumber
          FROM orders o
          JOIN orderdetails od
              ON o.orderNumber = od.orderNumber
          JOIN products p
              ON p.productCode = od.productCode
         )
         
    GROUP BY productName
    ORDER BY count DESC
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df


,productName,count
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [16]:
# Your code here
cur.execute(
    """
    SELECT e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
    FROM employees e
    JOIN offices o
    USING (officeCode)
    WHERE employeeNumber IN (SELECT salesRepEmployeeNumber
                             FROM employees e
                             JOIN customers c
                                 ON e.employeeNumber = c.salesRepEmployeeNumber
                             JOIN orders o
                                 ON o.customerNumber = c.customerNumber
                             JOIN orderdetails od
                                 ON od.orderNumber = od.orderNumber
                             WHERE productCode IN (SELECT productCode
                                                   FROM (SELECT productCode, COUNT(customerNumber) as count
                                                        FROM (SELECT DISTINCT productCode, customerNumber
                                                              FROM orders o
                                                              JOIN orderdetails od
                                                                  ON o.orderNumber = od.orderNumber
                                                             )
                                                        GROUP BY productCode
                                                        HAVING count < 20
                                                        )
                                                  ) 
                            )
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


In [17]:
# Your code here
cur.execute(
    """
    SELECT e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
    FROM employees e
    JOIN offices o
    USING (officeCode)
    WHERE employeeNumber IN (SELECT salesRepEmployeeNumber
                             FROM employees e
                             JOIN customers c
                                 ON e.employeeNumber = c.salesRepEmployeeNumber
                             JOIN orders o
                                 ON o.customerNumber = c.customerNumber
                             JOIN orderdetails od
                                 ON od.orderNumber = od.orderNumber
                             WHERE productCode IN (SELECT productCode
                                                        FROM (SELECT DISTINCT productCode, customerNumber
                                                              FROM orders o
                                                              JOIN orderdetails od
                                                                  ON o.orderNumber = od.orderNumber
                                                             )
                                                        GROUP BY productCode
                                                        HAVING COUNT(customerNumber) < 20
                                                    ) 
                            )
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [19]:
# Your code here
cur.execute(
    """
    SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(customerNumber) AS cust_count
    FROM employees e
    JOIN customers c
        ON e.employeeNumber = c.salesRepEmployeeNumber
    WHERE e.employeeNumber IN (SELECT salesRepEmployeeNumber
                                FROM customers
                                GROUP BY salesRepEmployeeNumber
                                HAVING AVG(creditLimit) > 15000
                              )
    ;
    """
           )

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,employeeNumber,firstName,lastName,cust_count
0,1165,Leslie,Jennings,100


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!